The following workbook is to scrape data from the url: <https://ca.milesplit.com/meets/494231-cvl-meet-3-ace-2022/results/846020/raw>

As our very first step, let's extract the race_id from the url programatically. This will be saved in our output dataframe and as part of the name of our output file.

In the space below, generate a variable called race_id and assign it the six digit value before the race name in the URL.  For example, in the url <https://ca.milesplit.com/meets/123456-example-race-name/results/999999/raw> the race_id should be 123456.  Verify that your code works by extracting the race_id from both the example url and your primary url. 

In [1]:
def extract_race_id(url):
    #define your function here!
    
    race_id = int(url.split('/meets/')[1].split('-')[0])
    return race_id

race_url = "https://ca.milesplit.com/meets/494231-cvl-meet-3-ace-2022/results/846020/raw"

race_id = extract_race_id(race_url)

# race_id = extract_race_id("https://ca.milesplit.com/meets/123456-example-race-name/results/999999/raw")

race_id

494231

Now lets process the HTML file! 

To get you started I've saved a file to the raw_html_files folder with example webpage (html) code. Change the file path to match the file path on your computer. Verify that the html file is being read correctly.

In [2]:
html_file_path = r"C:\Users\coleg\OneDrive\Documents\Econ Research Lab\raw_html_files\meet_494231.html"

In [3]:
import pandas as pd
from bs4 import BeautifulSoup
import os

with open(html_file_path, 'r', encoding='utf-8') as file:
    html_content = file.read()

soup = BeautifulSoup(html_content, 'html.parser')
soup.prettify()[:500]  # This will display the HTML content in a structured format

'<html lang="en" xmlns:="">\n <head>\n  <style>\n   <!----> <!--?lit$113130939$-->.osano-cm-window{font-family:Helvetica,Arial,Hiragino Sans GB,STXihei,Microsoft YaHei,WenQuanYi Micro Hei,Hind,MS Gothic,Apple SD Gothic Neo,NanumBarunGothic,sans-serif;font-size:16px;font-smooth:always;-webkit-font-smoothing:antialiased;-moz-osx-font-smoothingz:auto;display:block;left:0;line-height:1;position:absolute;top:0;width:100%;z-index:2147483638;--fade-transition-time:700ms;--slide-transition-time:400ms}.osano'

If the html displayed above, you have read in your file!

Next we need to identify the correct portion of the html file with the table that we want to scrape and format. Do so below.

In [4]:
def find_results_table(soup):
    results_div = soup.find("div", id="meetResultsBody")
    return str(results_div) if results_div else None  # or results_div.decode_contents()

# Call the function *outside* of its definition
table = find_results_table(soup)
print(table)  # print or inspect it


<div id="meetResultsBody">
<pre>  MS Boys 2 Mile (Middle School) Results
  Pl Athlete                   Yr Team                           Time
   1 Ethan AHUMANDA             7 Cobalt Institute of Mat     12:46.8 
   2 Michael SEARCY             7 Hesperia Christian          13:02.0 
   3 Dutch DEMKE                  Lucerne Valley              13:32.0 
   4 Cedric Liam KELLY          8 University Prep-Victorv     13:58.0 
   5 Joshua HARRIS              8 Academy for Academic Ex     13:58.8 
   6 Adrian LOPEZ               7 University Prep-Victorv     14:01.0 
   7 Omar LOPEZ                 7 University Prep-Victorv     14:01.3 
   8 Don WICKRAMASINGHE         7 Academy of Careers and      14:30.9 
   9 Tyler TIMMONS              6 Academy for Academic Ex     14:36.4 
  10 conner HACK                8 Academy of Careers and      14:50.8 
  11 Chisombiri OTUECHERE         Victor Valley Christian     14:56.2 
  12 Edgar ARCHILA                Lucerne Valley              15:09.2 
  13 

Next, transform the content in your html table or text into a pandas dataframe.  The pandas dataframe output must have the following column names:
- race_id
- race_url
- place
- athlete
- athlete_url
- grade
- team
- team_url
- finish
- point  

Get race_id from your generated variable above.  Get the race_url from the provided url.

If your text or table do not have the appropriate column names rename the columns or create the columns even if they are empty.

In [5]:
import re

def generate_dataframe(table):
    # Your code to convert the HTML table to a pandas DataFrame

    # Missing data is filled with None

    soup = BeautifulSoup(table, "html.parser")

    # Extract <pre> content
    pre = soup.find("pre")
    if not pre:
        raise ValueError("No <pre> tag found in the HTML.")

    text = pre.get_text("\n", strip=True)
    lines = text.splitlines()

    # First line is the race name
    race_name = lines[0] if lines else None
    if race_name and ' Results' in race_name:
        race_name = race_name.replace(' Results', '').strip()

    # Extract race_id from URL if available
    race_id = extract_race_id(race_url) if race_url else None

    # Only lines starting with a number (place)
    data_lines = [line for line in lines if re.match(r"^\s*\d+", line)]

    data = []
    for line in data_lines:
        # Regex to capture: place, athlete name, grade (optional), team, finish
        match = re.match(r"^\s*(\d+)\s+(.+?)\s+(\d+)?\s+(.+?)\s+(\d{1,2}:\d{2}\.\d)", line)
        if match:
            place = int(match.group(1))
            athlete = match.group(2).strip().title()
            grade = int(match.group(3)) if match.group(3) else pd.NA
            team = match.group(4).strip()
            finish = match.group(5)
            point = pd.NA  # Not in <pre>
            
            # Append row with all required columns
            data.append({
                "race_id": race_id,
                "race_url": race_url,
                "race_name": race_name,
                "place": place,
                "video": None,
                "athlete": athlete,
                "athlete_url": None,
                "grade": grade,
                "team": team,
                "team_url": None,
                "finish": finish,
                "point": point
            })

    # Convert to DataFrame and ensure column order
    columns = ['race_id', 'race_url', 'race_name', 'place', 'video', 'athlete', 
               'athlete_url', 'grade', 'team', 'team_url', 'finish', 'point']
    df = pd.DataFrame(data, columns=columns)
    return df

df = generate_dataframe(table)

df

,race_id,race_url,race_name,place,video,athlete,athlete_url,grade,team,team_url,finish,point
0,494231,https://ca.milesplit.com/meets/494231-cvl-meet...,MS Boys 2 Mile (Middle School),1,None,Ethan Ahumanda,None,7,Cobalt Institute of Mat,None,12:46.8,<NA>
1,494231,https://ca.milesplit.com/meets/494231-cvl-meet...,MS Boys 2 Mile (Middle School),2,None,Michael Searcy,None,7,Hesperia Christian,None,13:02.0,<NA>
2,494231,https://ca.milesplit.com/meets/494231-cvl-meet...,MS Boys 2 Mile (Middle School),3,None,Dutch Demke,None,<NA>,Lucerne Valley,None,13:32.0,<NA>
3,494231,https://ca.milesplit.com/meets/494231-cvl-meet...,MS Boys 2 Mile (Middle School),4,None,Cedric Liam Kelly,None,8,University Prep-Victorv,None,13:58.0,<NA>
4,494231,https://ca.milesplit.com/meets/494231-cvl-meet...,MS Boys 2 Mile (Middle School),5,None,Joshua Harris,None,8,Academy for Academic Ex,None,13:58.8,<NA>
5,494231,https://ca.milesplit.com/meets/494231-cvl-meet...,MS Boys 2 Mile (Middle School),6,None,Adrian Lopez,None,7,University Prep-Victorv,None,14:01.0,<NA>
6,494231,https://ca.milesplit.com/meets/494231-cvl-meet...,MS Boys 2 Mile (Middle School),7,None,Omar Lopez,None,7,University Prep-Victorv,None,14:01.3,<NA>
7,494231,https://ca.milesplit.com/meets/494231-cvl-meet...,MS Boys 2 Mile (Middle School),8,None,Don Wickramasinghe,None,7,Academy of Careers and,None,14:30.9,<NA>
8,494231,https://ca.milesplit.com/meets/494231-cvl-meet...,MS Boys 2 Mile (Middle School),9,None,Tyler Timmons,None,6,Academy for Academic Ex,None,14:36.4,<NA>
9,494231,https://ca.milesplit.com/meets/494231-cvl-meet...,MS Boys 2 Mile (Middle School),10,None,Conner Hack,None,8,Academy of Careers and,None,14:50.8,<NA>


In [6]:
df.isnull().sum()

race_id         0
race_url        0
race_name       0
place           0
video          41
athlete         0
athlete_url    41
grade           4
team            0
team_url       41
finish          0
point          41
dtype: int64

In [7]:
expected_columns = [
    'race_id', 'race_url', 'race_name', 'place', 'video', 'athlete', 'athlete_url',
    'grade', 'team', 'team_url', 'finish', 'point'
]

def verify_dataframe_columns(df, expected_columns):
    return list(df.columns) == expected_columns

def verify_dataframe_is_not_empty(df):
    return not df.empty

# Example usage:
df_columns_correct = verify_dataframe_columns(df, expected_columns)
df_is_not_empty = verify_dataframe_is_not_empty(df)

print("Columns are correct:", df_columns_correct)
print("DataFrame is not empty:", df_is_not_empty)

Columns are correct: True
DataFrame is not empty: True


Take time to verify the following about your dataframe: 
1. The dataframe is not empty.
2. Column names are correct and in the correct order
3. The data in the dataframe matches the data at the website url.

If anything is incorrect in your dataframe - iterate in the space above until it is correct!

Now generate the correct file name for your dataframe using the url your only input.  The URL for the HTML file is: <https://ca.milesplit.com/meets/494231-cvl-meet-3-ace-2022/results/846020/raw>

Below generate a file name in the format TABLETYPE_results_meet_MEETID.csv.  For example, individual_results_meet_123456.csv.  Options for TableType are individual or team for individual or team results.

In [8]:
def generate_filename(url, table_type):
    # Your code to generate the filename based on the URL and table type

    race_id = str(extract_race_id(url))

    filename = table_type + "_results_meet_" + race_id + ".csv"

    return filename

filename = generate_filename("https://ca.milesplit.com/meets/494231-cvl-meet-3-ace-2022/results/846020/raw", "individual")

filename

'individual_results_meet_494231.csv'

Finally, generate the correct file path, so that this csv saves in the folder 'output' in the 'research_assistant' folder.

In [9]:
file_path = r"C:\Users\coleg\OneDrive\Documents\Econ Research Lab\research_assistants copy\output"
file_path

'C:\\Users\\coleg\\OneDrive\\Documents\\Econ Research Lab\\research_assistants copy\\output'

In [10]:
if df_columns_correct and df_is_not_empty:
    print("DataFrame columns match the expected columns.")
    output_file_locatin = os.path.join(file_path, filename)
    df.to_csv(filename, index=False)
else:
    print("DataFrame columns do not match the expected columns or DataFrame is empty.")

DataFrame columns match the expected columns.
